In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_path = '/shared/TORCS_test.tfrecords'

feature = {'fast': tf.FixedLenFeature([], tf.float32),
    'dist_RR': tf.FixedLenFeature([], tf.float32),
    'dist_MM': tf.FixedLenFeature([], tf.float32),
    'dist_LL': tf.FixedLenFeature([], tf.float32),
    'toMarking_RR': tf.FixedLenFeature([], tf.float32),
    'toMarking_MR': tf.FixedLenFeature([], tf.float32),
    'toMarking_ML': tf.FixedLenFeature([], tf.float32),
    'toMarking_LL': tf.FixedLenFeature([], tf.float32),
    'dist_R': tf.FixedLenFeature([], tf.float32),
    'dist_L': tf.FixedLenFeature([], tf.float32),
    'toMarking_R': tf.FixedLenFeature([], tf.float32),
    'toMarking_M': tf.FixedLenFeature([], tf.float32),
    'toMarking_L': tf.FixedLenFeature([], tf.float32),
    'angle': tf.FixedLenFeature([], tf.float32),
    'image': tf.FixedLenFeature([], tf.string)}

# Create a list of filenames and pass it to a queue
filename_queue = tf.train.string_input_producer([data_path], num_epochs=1)
# Define a reader and read the next record
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
# Decode the record read by the reader
features = tf.parse_single_example(serialized_example, features=feature)
# Convert the image data from string back to the numbers
image = tf.decode_raw(features['image'], tf.uint8)

# Cast label data into float32
fast = tf.cast(features['fast'], tf.float32)

# rescale all targets to [0,1], numbers were taken from source code of DeepDriving
dist_RR = tf.cast(features['dist_RR'], tf.float32) / 95.0 + 0.12
dist_MM = tf.cast(features['dist_MM'], tf.float32) / 95.0 + 0.12
dist_LL = tf.cast(features['dist_LL'], tf.float32) / 95.0 + 0.12

toMarking_RR = tf.cast(features['toMarking_RR'], tf.float32) / 6.8752 - 0.48181
toMarking_MR = tf.cast(features['toMarking_MR'], tf.float32) / 6.25 + 0.02
toMarking_ML = tf.cast(features['toMarking_ML'], tf.float32) / 6.25 + 0.98
toMarking_LL = tf.cast(features['toMarking_LL'], tf.float32) / 6.8752 + 1.48181

dist_R = tf.cast(features['dist_R'], tf.float32) / 95.0 + 0.12
dist_L = tf.cast(features['dist_L'], tf.float32) / 95.0 + 0.12

toMarking_R = tf.cast(features['toMarking_R'], tf.float32) / 5.6249 - 0.34445
toMarking_M = tf.cast(features['toMarking_M'], tf.float32) / 6.8752 + 0.39091
toMarking_L = tf.cast(features['toMarking_L'], tf.float32) / 5.6249 + 1.34445

angle = tf.cast(features['angle'], tf.float32) / 1.1 + 0.5

# Reshape image data into the original shape
image = tf.reshape(image, [210, 280, 3])

# rescale from [0..255] to [-1..1]
image = tf.cast(image, tf.float32)
image = (image - 128.0) / 128.0

# Creates batches by randomly shuffling tensors
#images=1
images, fasts, dist_RRs, dist_MMs, dist_LLs, toMarking_RRs, toMarking_MRs, toMarking_MLs, toMarking_LLs, dist_Rs, dist_Ls, toMarking_Rs, toMarking_Ms, toMarking_Ls, angles = tf.train.shuffle_batch(
                    [image, fast, dist_RR, dist_MM, dist_LL, toMarking_RR, toMarking_MR, toMarking_ML,
                        toMarking_LL, dist_R, dist_L, toMarking_R, toMarking_M, toMarking_L, angle],
                        batch_size=16, capacity=30, num_threads=1, min_after_dequeue=10)


In [4]:
y_true = tf.concat([tf.expand_dims(fasts,1), tf.expand_dims(dist_RRs,1), tf.expand_dims(dist_MMs,1), tf.expand_dims(dist_LLs,1), 
                    tf.expand_dims(toMarking_RRs,1), tf.expand_dims(toMarking_MRs,1), tf.expand_dims(toMarking_MLs,1),
                    tf.expand_dims(toMarking_LLs,1), tf.expand_dims(dist_Rs,1), tf.expand_dims(dist_Ls,1), tf.expand_dims(toMarking_Rs,1),
                    tf.expand_dims(toMarking_Ms,1), tf.expand_dims(toMarking_Ls,1), tf.expand_dims(angles,1)], axis=1)

In [5]:
from alexnet import alexnet_v2


logits, endpoints = alexnet_v2(images,
               num_classes=1000,
               is_training=True,
               dropout_keep_prob=0.999,
               spatial_squeeze=True,
               scope='alexnet_v2')

y_pred = endpoints['alexnet_v2/fc8']
loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))

In [5]:
# from mobilenet_v1 import mobilenet_v1

# logits, endpoints = mobilenet_v1(images, #tf.cast(images, tf.float32),
#                  num_classes=1000,
#                  dropout_keep_prob=0.999,
#                  is_training=True,
#                  min_depth=8,
#                  depth_multiplier=0.75,
#                  conv_defs=None,
#                  prediction_fn=tf.contrib.layers.softmax,
#                  spatial_squeeze=True,
#                  reuse=None,
#                  scope='MobilenetV1')

# y_pred = endpoints['Logits']
# loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))

In [6]:
angle_residuals = []

toMarking_L_residuals = []
toMarking_M_residuals = []
toMarking_R_residuals = []

dist_L_residuals = []
dist_R_residuals = []

toMarking_LL_residuals = []
toMarking_ML_residuals = []
toMarking_MR_residuals = []
toMarking_RR_residuals = []

dist_LL_residuals = []
dist_MM_residuals = []
dist_RR_residuals = []

angles = []

with tf.Session() as sess:
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    # restore model
    saver = tf.train.Saver()

    saver.restore(sess, 'model_alex/139999.ckpt')
    
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for batch_index in range(2533/16):
        p, y = sess.run([y_pred, y_true])
        for j in range(y_true.shape[0]):
            residuals = p[j,:] - y[j,:]
            
            angles.append(y[j,13])
            angle_residuals.append(residuals[13-0] * 1.1)

            toMarking_L_residuals.append(residuals[13-1]*5.6249)
            toMarking_M_residuals.append(residuals[13-2]*6.8752)
            toMarking_R_residuals.append(residuals[13-3]*5.6249)

            dist_L_residuals.append(residuals[13-4]*95)
            dist_R_residuals.append(residuals[13-5]*95)

            toMarking_LL_residuals.append(residuals[13-6] * 6.8752)
            toMarking_ML_residuals.append(residuals[13-7] * 6.25)
            toMarking_MR_residuals.append(residuals[13-8] * 6.25)
            toMarking_RR_residuals.append(residuals[13-9] * 6.8752)

            dist_LL_residuals.append(residuals[13-10] * 95)
            dist_MM_residuals.append(residuals[13-11] * 95)
            dist_RR_residuals.append(residuals[13-12] * 95)

    # Stop the threads
    coord.request_stop()


INFO:tensorflow:Restoring parameters from model_alex/139999.ckpt


# Alexnet 140k iterations

In [7]:
print("angle: {}, to_LL: {}, to_ML: {}, to_MR: {}, to_RR: {}, to_L: {}, to_M: {}, to_R: {}".format(
    np.mean(np.abs(angle_residuals)), np.mean(np.abs(toMarking_LL_residuals)), np.mean(np.abs(toMarking_ML_residuals)),
    np.mean(np.abs(toMarking_MR_residuals)), np.mean(np.abs(toMarking_RR_residuals)), np.mean(np.abs(toMarking_L_residuals)),
    np.mean(np.abs(toMarking_M_residuals)), np.mean(np.abs(toMarking_R_residuals))))

angle: 0.0401853007634, to_LL: 0.172279581355, to_ML: 0.236961272449, to_MR: 0.219013178802, to_RR: 0.233891143186, to_L: 0.168281958686, to_M: 0.254165021174, to_R: 0.206914217719


In [8]:
print("dist_L: {}, dist_R: {}, dist_LL: {}, dist_MM: {}, dist_RR: {}".format(
    np.mean(np.abs(dist_L_residuals)), np.mean(np.abs(dist_R_residuals)),
    np.mean(np.abs(dist_LL_residuals)), np.mean(np.abs(dist_MM_residuals)), np.mean(np.abs(dist_RR_residuals))))

dist_L: 1.629121691, dist_R: 1.79990233818, dist_LL: 1.89498640785, dist_MM: 2.10143216414, dist_RR: 2.34845774295


# Alexnet 80k iterations

In [8]:
print("angle: {}, to_LL: {}, to_ML: {}, to_MR: {}, to_RR: {}, to_L: {}, to_M: {}, to_R: {}".format(
    np.mean(np.abs(angle_residuals)), np.mean(np.abs(toMarking_LL_residuals)), np.mean(np.abs(toMarking_ML_residuals)),
    np.mean(np.abs(toMarking_MR_residuals)), np.mean(np.abs(toMarking_RR_residuals)), np.mean(np.abs(toMarking_L_residuals)),
    np.mean(np.abs(toMarking_M_residuals)), np.mean(np.abs(toMarking_R_residuals))))

angle: 0.0416114217469, to_LL: 0.201853084625, to_ML: 0.236998392616, to_MR: 0.222831214793, to_RR: 0.239087398509, to_L: 0.175881728919, to_M: 0.273412398223, to_R: 0.247124108439


In [ ]:
print("dist_L: {}, dist_R: {}, dist_LL: {}, dist_MM: {}, dist_RR: {}".format(
    np.mean(np.abs(dist_L_residuals)), np.mean(np.abs(dist_R_residuals)),
    np.mean(np.abs(dist_LL_residuals)), np.mean(np.abs(dist_MM_residuals)), np.mean(np.abs(dist_RR_residuals))))

# Mobilenet 80k iterations

In [9]:
print("angle: {}, to_LL: {}, to_ML: {}, to_MR: {}, to_RR: {}, to_L: {}, to_M: {}, to_R: {}".format(
    np.mean(np.abs(angle_residuals)), np.mean(np.abs(toMarking_LL_residuals)), np.mean(np.abs(toMarking_ML_residuals)),
    np.mean(np.abs(toMarking_MR_residuals)), np.mean(np.abs(toMarking_RR_residuals)), np.mean(np.abs(toMarking_L_residuals)),
    np.mean(np.abs(toMarking_M_residuals)), np.mean(np.abs(toMarking_R_residuals))))

angle: 0.0491310474121, to_LL: 0.354534974728, to_ML: 0.383371908602, to_MR: 0.350780200649, to_RR: 0.412539151577, to_L: 0.307434286895, to_M: 0.432408641085, to_R: 0.352634229591


In [10]:
print("dist_L: {}, dist_R: {}, dist_LL: {}, dist_MM: {}, dist_RR: {}".format(
    np.mean(np.abs(dist_L_residuals)), np.mean(np.abs(dist_R_residuals)),
    np.mean(np.abs(dist_LL_residuals)), np.mean(np.abs(dist_MM_residuals)), np.mean(np.abs(dist_RR_residuals))))

dist_L: 9.00189648471, dist_R: 10.0919151594, dist_LL: 11.0662903272, dist_MM: 18.9990643161, dist_RR: 10.2342460218
